In [1]:
%save del_mols.py 1-1000
import argparse, sys, queue, os

parser = argparse.ArgumentParser()
parser.add_argument('-i', '--input', help = 'Input file')
parser.add_argument('-o', '--output', default = 'out.gro', help = 'Output filename')
parser.add_argument('-m', '--mols', nargs = '*', help = 'molecules to be deleted')
parser.add_argument('-n', '--nums', nargs = '*', type = int, help = 'number of molecules to be deleted')
parser.add_argument('-b', '--backward', action = 'store_true', help = 'delete from back if specified')
parser.add_argument('-v', action = 'store_true', help = 'Show details')

try:
    __IPYTHON__
    args = parser.parse_args('-i begin.gro -m TPA BF4 -n 2 2 -b'.split())
except NameError:
    args = parser.parse_args()
    
if len(args.mols) != len(args.nums):
    sys.exit('molecule name and number does not match!')

outname = args.output
while os.path.isfile(outname):
    outname = outname+'.dup...'

if args.v:
    print('Reading input file %s...\nWriting to file %s' %(args.input, outname))
    
    
mol_dict = {}
q_dict = {}
for i, j in zip(args.mols, args.nums):
    mol_dict[i] = j
    if args.backward:
        q_dict[i] =  queue.Queue()

def forward_delete(filename, outname):
    myfile = open(filename, 'r')
    outfile = open(outname, 'w')
    rec_dict = {}
    pre = None
    for i, line in enumerate(myfile):
        if i > 1:
            temp = line[5: 10].replace(' ', '')
            if temp in mol_dict and rec_dict.get(temp, 0) <= mol_dict[temp]:
                resid = int(line[:5])
                if resid != pre:
                    pre = resid
                    rec_dict[temp] =  rec_dict.get(temp, 0)+1
                if rec_dict[temp] <= mol_dict[temp]:
                    continue
        outfile.write(line)
    outfile.close()
    myfile.close()

# Use python queue module for the FIFO data structure
def backward_delete(filename, outname):
    myfile = open(filename, 'r')
    outfile = open(outname, 'w')
    rec_dict = {}
    pre = None
    for i, line in enumerate(myfile):
        if i > 1:
            temp = line[5: 10].replace(' ', '')
            if temp in mol_dict:
                q_dict[temp].put(line)
                resid = int(line[:5])
                if resid != pre:
                    pre = resid
                    rec_dict[temp] =  rec_dict.get(temp, 0)+1
                if rec_dict[temp] > mol_dict[temp]:
                    outfile.write(q_dict[temp].get())
                    continue
        outfile.write(line)
    outfile.close()
    myfile.close()

if args.backward:
    backward_delete(args.input, outname)
else:
    forward_delete(args.input, outname)

The following commands were written to file `del_mols.py`:
get_ipython().run_line_magic('save', 'del_mols.py 1-1000')
import argparse, sys, queue, os

parser = argparse.ArgumentParser()
parser.add_argument('-i', '--input', help = 'Input file')
parser.add_argument('-o', '--output', default = 'out.gro', help = 'Output filename')
parser.add_argument('-m', '--mols', nargs = '*', help = 'molecules to be deleted')
parser.add_argument('-n', '--nums', nargs = '*', type = int, help = 'number of molecules to be deleted')
parser.add_argument('-b', '--backward', action = 'store_true', help = 'delete from back if specified')
parser.add_argument('-v', action = 'store_true', help = 'Show details')

try:
    __IPYTHON__
    args = parser.parse_args('-i begin.gro -m TPA BF4 -n 2 2 -b'.split())
except NameError:
    args = parser.parse_args()
    
if len(args.mols) != len(args.nums):
    sys.exit('molecule name and number does not match!')

outname = args.output
while os.path.isfile(outname):
    outnam